# OperatorAware
## Urgency detection for emergency calls
mitchellpkt@protonmail.com

OperatorAware evaluates 911 calls in real-time to assess the nature and direness of the emergency being reported.

As the human operator collects information, OperatorAware converts the audio stream to text, and scans for keywords indicating critical situations (e.g. the word "gun" indicating that there is likely a weapon involved). As the call continues, OperatorAware updates the likelihood of a situation involving:
-  Weapons (e.g. inferred from use of words "gun" and "shot" by a caller)
-  Medical emergencies 
-  Vehicle accidents
-  etc

## Example
Caller reports: **"My neighbor shot my leg, It's bleeding everywhere! He didn't mean to discharge the gun"** 

OperatorAware uses the words {"shot", "bleeding", "gun"} to identify that this is a **medical emergency involving a weapon.**

## Motivation
There have been multiple reported instances of emergency operators hanging up on 911 callers during crises. 

For example, one operator who stated "You could deal with it yourself. I’m not gonna deal with this, okay?” before hanging up on a caller that was performing CPR on a shooting victim ([Washington Post](https://www.washingtonpost.com/news/post-nation/wp/2015/07/29/deal-with-it-yourself-911-dispatcher-tells-panicked-caller-with-dying-friend/?utm_term=.eea24de1e5f3)). 

Another 911 dispatcher, infamous for ending a call with "Ain't nobody got time for this," hung up on reports of street racing and armerd robberies (different [Washington Post](https://www.washingtonpost.com/news/post-nation/wp/2018/04/19/911-dispatcher-jailed-houston-woman-hung-up-on-thousands-of-callers/?noredirect=on&utm_term=.b4bb2b6e8f37) article). In this case, later analysis showed that thousands of calls shorter than 20 seconds were attributed to her hanging up. She stated that she hung up because at those times she did not want to talk to anyone ([BBC](http://www.bbc.com/news/world-us-canada-43822504)).

While 911 callers may hang up at any time, in the vast majority of cases, **the 911 dispatcher should not be the party to terminate the call during a crisis.** If an operator is the party to end a call that has been ranked as likely-severe by the OperatorAware, this should trigger quality assurance review. The latter dispatcher mentioned above hung up on thousands of calls over multiple years. With OperatorAware, she would have come under close scrutiny by the end of her first week

## Use case scale
The [Washington Post](https://www.washingtonpost.com/news/post-nation/wp/2018/04/19/911-dispatcher-jailed-houston-woman-hung-up-on-thousands-of-callers/?noredirect=on&utm_term=.d2fbe079869a) notes: *This single consolidated center for 911 calls opened in 2003 and handles millions of calls every year, according to the Chronicle, or 9,000 a day. Two-thirds of those calls aren’t true emergencies*

*The rest involve people in dire need.*



# Preparation

## Enter name of file to be transcribed

In [2]:
sound_filename = 'brooklyn_murder_16signed_short.raw'

## Select dictionary

In [3]:
# proof-of-concept dictionary. For actual implementation, would need multiple formes e.g. 'shoot' and 'shot'
DangerWords = {'weaponWords': ['knife', 'gun', 'weapon', 'shoot', 'shot', 'armed'],
        'medicalWords': ['heart','stroke','breathing','unconscious','collapsed'],
        'vehicleWords': ['crash', 'accident','airbag'],
        'miscWords': ['violent','suicidal', 'suicide']};

## Import/install libraries

In [13]:
#!pip install --upgrade google-cloud-speech

import io
import os
from google.cloud.speech import types, enums

## Load in the audio file

In [14]:
### Form filename
file_name = os.path.join(
     os.getcwd(),
    'Real_911Calls',
    sound_filename)

### Load the audio into memory
with io.open(file_name, 'rb') as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)

config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='en-US')


# Try to follow instruction set 1:
https://stackoverflow.com/questions/38703853/how-to-use-google-speech-recognition-api-in-python

## Import libraries

In [6]:
import argparse
import base64
import json

from googleapiclient import discovery
import httplib2
from oauth2client.client import GoogleCredentials

## Set discovery URL

In [15]:
DISCOVERY_URL = ('https://{api}.googleapis.com/$discovery/rest?'
                 'version={apiVersion}')

## Define get_speech_service

In [16]:
def get_speech_service():
    credentials = GoogleCredentials.get_application_default().create_scoped(
        ['https://www.googleapis.com/auth/cloud-platform'])
    http = httplib2.Http()
    credentials.authorize(http)

    return discovery.build(
        'speech', 'v1beta1', http=http, discoveryServiceUrl=DISCOVERY_URL)

## Define main_speech

In [17]:
def main_speech(speech_file): # I changed the name for Jupyter
    """Transcribe the given audio file.

    Args:
        speech_file: the name of the audio file.
    """
    with open(speech_file, 'rb') as speech:
        speech_content = base64.b64encode(speech.read())

    service = get_speech_service()
    service_request = service.speech().syncrecognize(
        body={
            'config': {
                'encoding': 'LINEAR16',  # raw 16-bit signed LE samples
                'sampleRate': 16000,  # 16 khz
                'languageCode': 'en-US',  # a BCP-47 language tag
            },
            'audio': {
                'content': speech_content.decode('UTF-8')
                }
            })
    response = service_request.execute()
    print(json.dumps(response))

In [18]:
main_speech(file_name)

{}


# Try to follow instruction set 2:
https://cloud.google.com/speech-to-text/docs/quickstart-client-libraries

In [ ]:
# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

## Instantiate a client

In [ ]:
!export GOOGLE_APPLICATION_CREDENTIALS="/home/m/Dropbox/Projects/OperatorAware/GitPath/operatoraware/OperatorAware-5f653aaf3399.json"
client = speech.SpeechClient()

## Transcribe the call

In [ ]:
# Detect speech in the audio file
response = client.recognize(config, audio)

## Display transcription

In [ ]:
for result in response.results:
    print('Transcript: {}'.format(result.alternatives[0].transcript))